In [1]:
import sys
sys.path.append('../functionall/')
import pandas as pd
import numpy as np
import random
import CNLS, CSVR, toolbox
from constant import CET_ADDI, FUN_PROD, OPT_LOCAL, RTS_VRS
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

In [2]:
def yhat(alpha, beta, x_test):

    # compute yhat for each testing observation
    yhat = np.zeros((len(x_test),))
    for i in range(len(x_test)):
        yhat[i] = (alpha + np.matmul(beta, x_test[i].T)).min(axis=0)

    return yhat

In [18]:
data = pd.read_csv('Boston.csv')


# #Remove MEDV outliers (MEDV = 50.0)
# idx = data.loc[data['MEDV'] >= 50].index
# data = data.drop(idx)
data = data.head(50)

x = data.loc[:, ['NOX', 'DIS', 'RM', 'PTRATIO']]
y = data['MEDV']
for col in ['NOX', 'DIS']:
    x[col] = np.log1p(x[col])
x = np.array(x)
y = np.array(np.log1p(y))


In [4]:
alpha, beta, ksia, ksib = CSVR.CSVR(y, x, epsilon=0.1, u=2)

In [5]:
alpha

array([3.05510897, 3.02395607, 3.1684474 , 3.19564049, 3.25548116,
       3.13356355, 3.03071135, 3.04700907, 3.02601715, 3.04887025,
       3.07769448, 3.04887014, 3.02688905, 3.03440177, 3.03440176,
       3.03440176, 3.03440176, 3.03440174, 3.03440172, 3.03440174])

In [6]:
beta

array([[-2.75307093e-11,  8.09449243e-02,  8.24887609e-02,
         1.00779555e-01],
       [ 1.59379777e-08,  9.99300410e-02,  1.30845900e-01,
         4.22614155e-02],
       [ 2.03450408e-08,  7.94137828e-02,  1.60129654e-07,
         3.35849410e-02],
       [ 6.11739858e-05,  5.56924048e-05,  6.12408347e-02,
         5.62877425e-05],
       [ 6.00466426e-05,  5.57026803e-05,  5.54417752e-05,
         5.62524180e-05],
       [ 6.40436296e-05,  5.56743254e-05,  1.30845900e-01,
         5.63338384e-05],
       [ 4.69643154e-10,  2.24813637e-02,  1.30845915e-01,
         1.39189601e-01],
       [-1.12397776e-10,  8.61432815e-03,  1.18537878e-01,
         1.49731309e-01],
       [ 1.23827015e-09,  2.49887675e-09,  2.05542258e-01,
         1.08623213e-01],
       [ 1.21082871e-10,  1.19540729e-08,  1.30845828e-01,
         1.40483039e-01],
       [-7.67704255e-10,  2.53094396e-09,  6.12409472e-02,
         1.95587256e-01],
       [ 1.19220267e-10,  1.01595382e-07,  1.30845904e-01,
      

In [19]:
# CNLS
kfold = 5
error_all = []
for k in range(kfold):
    print("Fold", k, "\n")
    i_mix = random.sample(range(len(y)), k=len(y))

    # divide up i.mix into K equal size chunks
    m = len(y) // kfold
    i_kfold = [i_mix[i:i+m] for i in range(0, len(i_mix), m)]

    i_tr = toolbox.index_tr(k, i_kfold)
    i_val = i_kfold[k]

    # training predictors, training responses
    x_tr = x[i_tr, :]  
    y_tr = y[i_tr]   

    # validation predictors, validation responses
    x_val = x[i_val, :]
    y_val = y[i_val] 

    model = CNLS.CNLS(y_tr, x_tr, z=None, cet= CET_ADDI, fun= FUN_PROD, rts= RTS_VRS)
    model.optimize(OPT_LOCAL)
    alpha, beta = model.get_alpha(), model.get_beta()

    error_all.append(np.mean((yhat(alpha, beta, x_val) - y_val)**2))

mse = np.mean(np.array(error_all), axis=0)


Fold 0 

Fold 1 

Fold 2 

Fold 3 

Fold 4 



In [20]:
mse

0.13249620181578178